In [1]:
pwd

'/Users/josephboen/Desktop/Stanford/nyt_puzzles'

In [2]:
ls

README.md           WordleHelper.ipynb  spelling_bee.py~    wordle-La.txt
SpellingBee.ipynb   spelling_bee.py     unigram_freq.csv    words_alpha.txt


In [47]:
from tqdm import tqdm

In [3]:
def load_words(fp: str = 'wordle-La.txt') -> set:
    '''
    Loads list of all english words that contain only letters
    
    Params:
        fp: file path to wordle word list
    '''
    
    with open(fp) as word_file:
        valid_words = set(word_file.read().split())

    return valid_words

In [4]:
all_words = load_words()

In [25]:
def check_words(all_words: set, gray: set, green: dict, yellow: dict) -> set:
    '''
    Filters list of words to see which match wordle conditions
    
    Params:
        all_words: words to check
        gray: letters to exclude
        green: letters to include with correct positions
        yellow: letters to include with incorrect positions
    
    Returns: 
        set of valid words
    '''

    valid_words = set()
    for word in all_words:

        letters = set(word)

        if len(gray.intersection(letters)) == 0:
            exclude_check = True
        else:
            exclude_check = False

        yellow_check = True
        for pos, char in yellow.items():  
            if word[pos] == char or char not in letters:
                yellow_check = False

        green_check = True
        for pos, char in green.items():
            if word[pos] != char:
                green_check = False

        if exclude_check and yellow_check and green_check:
            valid_words.add(word)
            
    return valid_words

def parse_board(board: list):
    '''
    Parses wordle board
    
    Note: Gray -> X, Yellow -> Y, Green -> G
    
    Example:
        first guess: "adieu" yields "gray, yellow, gray, gray, yellow"
        second guess: "bound" yields "gray, green, green, green, green"
        
        then board = [("adieu", "XYXXY"), ("bound", "XGGGG")]
    
    Param:
        board: list of tuples containing guesses and outputs
        
    Returns:
        conditions: dictionary of conditions
    
    '''
    
    conditions = {"gray": set(), 
                  "green": {}, 
                  "yellow": {}}


    for guess, output in board:
        assert len(guess) == len(output), ("guess and output are different lengths!")
        assert set(output).issubset(set("XYG")), ("outputs must be X,Y, or G!")

        for idx, (letter, color) in enumerate(zip(guess, output)):
            if color == "X":
                conditions["gray"].add(letter)

            if color == "Y":
                conditions["yellow"][idx] = letter

            if color == "G":
                conditions["green"][idx] = letter
                
    return conditions

In [36]:
gray = set()
green = {}
yellow = {}

In [37]:
valid_words = check_words(all_words, gray, green, yellow)
len(valid_words)

2315

In [44]:
gray = set("aie")
green = {}
yellow = {1:"d", 4:"u"}

In [45]:
valid_words = check_words(all_words, gray, green, yellow)
len(valid_words)

30

In [48]:
for i in tqdm(range(1000)):
    valid_words = check_words(all_words, gray, green, yellow)

100%|██████████| 1000/1000 [00:02<00:00, 384.50it/s]


In [46]:
%timeit check_words(all_words, gray, green, yellow)

2.74 ms ± 48.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [41]:
gray = set("aieb")
green = {1:"o", 2:"u", 3:"n", 4:"d"}
yellow = {1:"d", 4:"u"}

In [42]:
valid_words = check_words(all_words, gray, green, yellow)
len(valid_words)

7

In [43]:
valid_words

{'found', 'hound', 'mound', 'pound', 'round', 'sound', 'wound'}

In [52]:
gray_line = ["a_ie_", "b____"]
green_line = ["", "_ound"]
yellow_line = ["_d__u", ""]

def parse_lines(gray_line, green_line, yellow_line):
    
    gray = set()
    for line in gray_line:
        for char in list(line):
            if char.isalpha():
                gray.add(char)
                
    green = {}
    for line in green_line:
        for idx, char in enumerate(list(line)):
            if char.isalpha():
                green[idx] = char
                
    yellow = {}
    for line in yellow_line:
        for idx, char in enumerate(list(line)):
            if char.isalpha():
                yellow[idx] = char
            
    return gray, green, yellow

parse_lines(gray_line, green_line, yellow_line)

({'a', 'b', 'e', 'i'}, {1: 'o', 2: 'u', 3: 'n', 4: 'd'}, {1: 'd', 4: 'u'})

In [62]:
board = [("adieu", "XYXXY"), ("bound", "XGGGG")]

#guess = "adieu"
#output = "XYXXG"

In [94]:
def parse_board(board: list):
    '''
    Parses wordle board
    
    Note: Gray -> X, Yellow -> Y, Green -> G
    
    Example:
        first guess: "adieu" yields "gray, yellow, gray, gray, yellow"
        second guess: "bound" yields "gray, green, green, green, green"
        
        this can be inputted as in ["adieu", "XYXXY","bound", "XGGGG"]
    
    Param:
        board: list of tuples containing guesses and outputs
        
    Returns:
        conditions: dictionary of conditions
    
    '''
    
    assert len(board)%2 == 0, ("unequal number of guesses and outputs!")
    
    reshaped_board = np.asarray(board).reshape((2,int(len(board)/2)))
    
    conditions = {"gray": set(), 
                  "green": {}, 
                  "yellow": {}}
    
    for guess, output in reshaped_board:
        assert len(guess) == len(output), ("guess and output are different lengths!")
        assert set(output).issubset(set("XYG")), ("outputs must be X,Y, or G!")

        for idx, (letter, color) in enumerate(zip(guess, output)):
            if color == "X":
                conditions["gray"].add(letter)

            if color == "Y":
                conditions["yellow"][idx] = letter

            if color == "G":
                conditions["green"][idx] = letter
                
    return conditions

In [95]:
parse_board(["adieu", "XYXXY", "board", "XGGGG"])

{'gray': {'a', 'b', 'e', 'i'},
 'green': {1: 'o', 2: 'a', 3: 'r', 4: 'd'},
 'yellow': {1: 'd', 4: 'u'}}

In [64]:
gray

{'a', 'b', 'e', 'i'}

In [65]:
green

{1: 'o', 2: 'u', 3: 'n', 4: 'd'}

In [66]:
yellow

{1: 'd', 4: 'u'}

In [68]:
parse_board(board)

{'gray': {'a', 'b', 'e', 'i'},
 'green': {1: 'o', 2: 'u', 3: 'n', 4: 'd'},
 'yellow': {1: 'd', 4: 'u'}}

In [71]:
help_message = (
    
    '''
    
    wordle board
    
    Note: Gray -> X, Yellow -> Y, Green -> G

    Example:
        first guess: "adieu" yields "gray, yellow, gray, gray, yellow"
        second guess: "bound" yields "gray, green, green, green, green"

        then board = [("adieu", "XYXXY"), ("bound", "XGGGG")]
    
    '''
    )

In [72]:
print(help_message)


    
    wordle board
    
    Note: Gray -> X, Yellow -> Y, Green -> G

    Example:
        first guess: "adieu" yields "gray, yellow, gray, gray, yellow"
        second guess: "bound" yields "gray, green, green, green, green"

        then board = [("adieu", "XYXXY"), ("bound", "XGGGG")]
    
    


In [87]:
import numpy as np

In [112]:
#board = np.asarray(["adieu", "XYXXY", "bound", "XGGGG"])
#board = np.asarray(["adieu", "XYXXY"])

board = ["adieu", "XYXXY"]

reshaped_board = [(board[i], board[i + 1]) for i in range(0, len(board), 2)]

reshaped_board

[('adieu', 'XYXXY')]

In [109]:
reshaped_board = board.reshape((2,int(len(board)/2)))
reshaped_board.shape

(2, 1)

In [110]:
for guess, output in reshaped_board:
    print(guess, output)

ValueError: not enough values to unpack (expected 2, got 1)

In [111]:
reshaped_board

array([['adieu'],
       ['XYXXY']], dtype='<U5')